# CNN+LSTM の Encoder-Decoderモデル

In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import pickle
from PIL import Image
import matplotlib.pyplot as plt
from SSRW_encoder_decoder import SSRWEncoderDecoder
from Trainer import Trainer, SpeakingDataset, collate_fn, DatasetHolder

## ハイパーパラメータの設定

In [2]:
BATCH_SIZE = 16
lstm_hidden_dim = 128
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


In [3]:
EPOCHS = 20

## データセット、データロードの設定

In [4]:
datasetHolder = DatasetHolder()

In [5]:
dataset_train, dataset_valid, dataset_test = datasetHolder.get_all_datasets()

In [6]:
dataLoader_train = DataLoader(dataset_train,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              collate_fn=collate_fn)
dataLoader_valid = DataLoader(dataset_valid,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              collate_fn=collate_fn)
dataLoader_test = DataLoader(dataset_test,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              collate_fn=collate_fn)

In [7]:
len(datasetHolder.dict)

46

## モデルのインスタンス化

In [8]:
model = SSRWEncoderDecoder(vocab_size=len(datasetHolder.dict)+4,
                           hidden_dim_encoder=lstm_hidden_dim,
                           hidden_dim_decoder=lstm_hidden_dim,
                           device=device)

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [10]:
trainer = Trainer(model=model,
                  dataloader_train=dataLoader_train,
                  dataloader_valid=dataLoader_valid,
                  criterion=criterion,
                  optimizer=optimizer,
                  device=device)

In [11]:
save_path = './enocder_decoder.pth'
torch.save(model.to('cpu').state_dict(), save_path)

In [12]:
trainer.train(epochs=EPOCHS, batch_size=BATCH_SIZE, hidden_dim=lstm_hidden_dim)

  0%|          | 0/20 [00:00<?, ?it/s]

train


RuntimeError: stack expects a non-empty TensorList

## Save Model

In [ ]:
save_path = './enc_dec.pth'
torch.save(music_arranger.to('cpu').state_dict(), save_path)

## Plot and Save Loss and Accuracy

In [ ]:
import pandas as pd
loss_dict = {'train_loss': trainer.train_loss_list,
             'valid_loss': trainer.valid_loss_list,
             'train_acc': trainer.train_acc_list,
             'valid_acc': trainer.valid_acc_list}
df = pd.DataFrame(loss_dict)
df.to_csv('./enc_dec_losses.csv')

In [ ]:
df = pd.read_csv('./enc_dec_losses.csv')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
fig, ax1 = plt.subplots()
x = np.array([i for i in range(EPOCH)])
ax1.set_xlabel('epochs')
ax1.plot(x, df['train_loss'], label='train_loss')
ax1.plot(x, df['valid_loss'], label='valid_loss')
ax1.legend()

ax2 = ax1.twinx()
ax2.plot(x, df['train_acc'], label='train_acc')
ax2.plot(x, df['valid_acc'], label='valid_acc')
ax2.legend()

ax1.set_ylabel('loss')
ax2.set_ylabel('acc')
plt.savefig('./graph.png')
plt.show()

## Load Model

In [ ]:
save_path = './enc_dec.pth'
model = SSRWEncoderDecoder
model.load_state_dict(torch.load(save_path))